In [361]:
import sys
sys.path.append('..')

In [362]:
db_folder = 'db'
db_name = 'test_db.db'
DB_CONNECTION_STRING = f'sqlite:///{db_folder}/{db_name}'
DB_CONNECTION_STRING

'sqlite:///db/test_db.db'

In [363]:
import json
from sqlalchemy.orm import sessionmaker
from sqlalchemy import and_

In [364]:
from model.model import db_connect, create_table, TJobPrim, TRawJobDesc, TRawCompDesc

In [365]:
engine = db_connect(DB_CONNECTION_STRING)
create_table(engine)
Session = sessionmaker(bind=engine)

>>> DB_CONNECTION_STRING: sqlite:///db/test_db.db


In [366]:
session = Session()

In [367]:
r = session.query(TRawJobDesc).all()

In [369]:
r = session.query(TRawJobDesc).all()
j = r[0].search_job_json
search_job_dict = json.loads(j)
d = search_job_dict
# d

In [333]:
import re

def _get_job_id_from_link(link):
    try:
        return re.search('job\/(.*)\?', link).group(1)
    except:
        print(f'There is no JOB_ID. (link: {link})')
        return None

In [349]:
# search job json
d = search_job_dict

job_type = d['jobType'] # code
job_no = d['jobNo']
job_name = d['jobName']
job_role = d['jobRole'] # code
job_addr_dist = d['jobAddrNoDesc']
job_desc = d['description']
edu = d['optionEdu']
work_exp = d['period'] # int
salary_min = d['salaryLow'] # int
salary_max = d['salaryHigh'] # int
salary_desc = d['salaryDesc']
salary_type = d['s10'] # code 10:面議 20:論件計酬 30:時薪 50:月薪
appear_date = d['appearDate'] # int
job_tag = '、'.join(d['tags'])
landmark_tag = d['landmark']
job_link = 'https:' + d['link']['job'] if 'job' in d['link'].keys() else ''
job_id = _get_job_id_from_link(job_link)
lon = d['lon'][:11]
lat = d['lat'][:11]


# analysis
apply_count = int(d['applyCnt'])

'0'

In [ ]:
# utils
def join_dict_item_in_list(l: list, key: str):
    dict_item_list = [d[key] for d in l]
    return '、'.join(dict_item_list)


def join_list_element(l: list):
    return '、'.join(l)

In [210]:
def _get_accept_role_string(d):
    l = d['data']['condition']['acceptRole']['role']
    return join_dict_item_in_list(l, 'description')


def _get_disaccept_role_string(d):
    l = d['data']['condition']['acceptRole']['disRole']['disability']
    return join_dict_item_in_list(l, 'type')


def _get_major_string(d):
    l = d['data']['condition']['major']
    return join_list_element(l)


def _get_language_string(d):
    l = d['data']['condition']['language']
    return join_dict_item_in_list(l, 'language')


def _get_local_language_string(d):
    l = d['data']['condition']['localLanguage']
    return join_dict_item_in_list(l, 'language')


def _get_specialty_string(d):
    l = d['data']['condition']['specialty']
    return join_dict_item_in_list(l, 'description')


def _get_skill_string(d):
    l = d['data']['condition']['skill']
    return join_dict_item_in_list(l, 'description')


def _get_certificate_string(d):
    l = d['data']['condition']['certificate']
    return join_list_element(l)


def _get_driver_license_string(d):
    l = d['data']['condition']['driverLicense']
    return join_list_element(l)


def _get_other_string(d):
    return d['data']['condition']['other']


def _get_job_category_string(d):
    l = d['data']['jobDetail']['jobCategory']
    return join_dict_item_in_list(l, 'description')


def _get_manage_resp_string(d):
    return d['data']['jobDetail']['manageResp']


def _get_business_trip_string(d):
    return d['data']['jobDetail']['businessTrip']


def _get_work_period_string(d):
    return d['data']['jobDetail']['workPeriod']


def _get_vacation_policy_string(d):
    return d['data']['jobDetail']['vacationPolicy']


def _get_start_working_day_string(d):
    return d['data']['jobDetail']['startWorkingDay']


def _get_hire_type_string(d):
    ''' code
    0: 公司自聘
    1: 代徵
    '''
    return str(d['data']['jobDetail']['hireType'])


def _get_delegate_recruit_string(d):
    return d['data']['jobDetail']['delegatedRecruit']


def _get_need_emp_string(d):
    return d['data']['jobDetail']['needEmp']

In [212]:
# job_desc_json
d = job_desc_dict

accept_role = _get_accept_role_string(d)
disaccept_role = _get_disaccept_role_string(d)
major = _get_major_string(d)
lang = _get_language_string(d)
local_lang = _get_local_language_string(d)
speciality = _get_specialty_string(d)
skill = _get_skill_string(d)
cert = _get_certificate_string(d)
driver_license = _get_driver_license_string(d)
other = _get_other_string(d)
job_cat = _get_job_category_string(d)
manage_resp = _get_manage_resp_string(d)
business_trip = _get_business_trip_string(d)
work_period = _get_work_period_string(d)
vacation_policy = _get_vacation_policy_string(d)
start_work_day = _get_start_working_day_string(d)
hire_type = _get_hire_type_string(d)
delegate_recruit = _get_delegate_recruit_string(d)


# analysis
need_count = _get_need_emp_string(d)

In [315]:
from datetime import datetime
import pandas as pd


def _convert_analysis_dict_to_format_json(raw_analysis_dict):
    stat_dict = {key: raw_analysis_dict[key] for key in raw_analysis_dict.keys() if key.isdigit()}
    stat_json = json.dumps(stat_dict, ensure_ascii=False)
    stat_df = pd.read_json(stat_json).T
    format_json = stat_df.to_json(force_ascii=False)
    return format_json


def _get_analysis_json_by_type_name(raw_dict, key):
    raw_analysis_dict = raw_dict[key]
    analysis_json = _convert_analysis_dict_to_format_json(raw_analysis_dict)
    return analysis_json


def _get_analysis_total(raw_dict):
    ''' TODO: use DFS to find first 'total' value. Same thing for "_get_update_date"
    '''
    for type_name in analysis_type_list:
        if 'total' in raw_dict[type_name].keys():
            total = raw_dict[type_name]['total']
            return int(total)
    return -1
    

def _get_update_date(raw_dict):
    for type_name in analysis_type_list:
        if 'update_time' in raw_dict[type_name].keys():
            update_time_string = raw_dict[type_name]['update_time']
            update_date = datetime.strptime(update_time_string, '%Y-%m-%d %H:%M:%S')
            update_date = int(update_date.strftime('%Y%m%d'))
            return update_date
    return -1

In [314]:
# job_analysis_json
d = job_analysis_dict

analysis_type_list = ['sex', 'edu', 'yearRange', 'exp', 'language', 'major', 'skill', 'cert']
total = _get_analysis_total(d)
update_date = _get_update_date(d)
sex = _get_analysis_json_by_type_name(d, 'sex')
edu = _get_analysis_json_by_type_name(d, 'edu')
year_range = _get_analysis_json_by_type_name(d, 'yearRange')
work_exp = _get_analysis_json_by_type_name(d, 'exp')
lang = _get_analysis_json_by_type_name(d, 'language')
major = _get_analysis_json_by_type_name(d, 'major')
skill = _get_analysis_json_by_type_name(d, 'skill')
cert = _get_analysis_json_by_type_name(d, 'cert')


In [348]:
cnt = 0
for r in session.query(TRawJobDesc).all():
    j = r.search_job_json
    job_desc_dict = json.loads(j)
    d = job_desc_dict
    if d['jobType'] == '2':
        print(d)
        print()
        if cnt > 10:
            break

{'jobType': '2', 'jobNo': '11612382', 'jobName': '網頁爬蟲工程師', 'jobNameSnippet': "網頁<em class='b-txt--highlight'>爬蟲</em>工程師", 'jobRole': '1', 'jobRo': '1', 'jobAddrNo': '6001008007', 'jobAddrNoDesc': '台中市西屯區', 'jobAddress': '市政北二路236號35樓之1', 'description': '網站[[[爬蟲]]]、資料輸出、API串接', 'optionEdu': '大學', 'period': '00', 'periodDesc': '經歷不拘', 'applyCnt': '00001', 'applyDesc': '0~5人應徵', 'custNo': '130000000012043', 'custName': '凡谷興業有限公司', 'coIndustry': '1001001006', 'coIndustryDesc': '其它軟體及網路相關業', 'salaryLow': '0030000', 'salaryHigh': '0060000', 'salaryDesc': '月薪30,000~60,000元', 's10': '50', 'appearDate': '20201104', 'appearDateDesc': '11/04', 'optionZone': '2101376', 'isApply': '0', 'applyDate': '', 'isSave': '0', 'descSnippet': "網站<em class='b-txt--highlight'>爬蟲</em>、資料輸出、API串接", 'tags': [], 'landmark': '', 'link': {'applyAnalyze': '//www.104.com.tw/jobs/apply/analysis/6ww66?channel=104rpt&jobsource=apply_analyze', 'job': '//www.104.com.tw/job/6ww66?jobsource=n104bank2', 'cust': '//www.104.com

In [341]:
d

{'jobType': '2',
 'jobNo': '10402767',
 'jobName': '軟體工程師',
 'jobNameSnippet': '軟體工程師',
 'jobRole': '1',
 'jobRo': '1',
 'jobAddrNo': '6003001001',
 'jobAddrNoDesc': '日本',
 'jobAddress': '東京都千代田区岩本町3-3-3サザンビル3階',
 'description': '進駐日本各大手企業現場進行軟體的開發和測試工作（有日語基礎或従事面向日本程式開發的経験者優先考慮）',
 'optionEdu': '專科',
 'period': '03',
 'periodDesc': '2年以上',
 'applyCnt': '00001',
 'applyDesc': '0~5人應徵',
 'custNo': '130000000116679',
 'custName': 'LYC株式会社',
 'coIndustry': '1001001002',
 'coIndustryDesc': '電腦軟體服務業',
 'salaryLow': '0060000',
 'salaryHigh': '0140000',
 'salaryDesc': '月薪60,000~140,000元',
 's10': '50',
 'appearDate': '20191226',
 'appearDateDesc': '12/26',
 'optionZone': '12288',
 'isApply': '0',
 'applyDate': '',
 'isSave': '0',
 'descSnippet': '進駐日本各大手企業現場進行軟體的開發和測試工作（有日語基礎或従事面向日本程式開發的経験者優先考慮）',
 'tags': ['員工95人'],
 'landmark': '',
 'link': {'applyAnalyze': '//www.104.com.tw/jobs/apply/analysis/66ytr?channel=104rpt&jobsource=apply_analyze',
  'job': '//www.104.com.tw/job/66ytr?jobsource=2018

20201107

In [213]:
r = session.query(TRawJobDesc).all()
j = r[0].job_desc_json
job_desc_dict = json.loads(j)
d = job_desc_dict
# d

In [265]:
j = r[0].job_analysis_json
job_analysis_dict = json.loads(j)
d = job_analysis_dict
d.keys()

dict_keys(['sex', 'edu', 'yearRange', 'exp', 'language', 'major', 'skill', 'cert'])

In [46]:
t='''    crawl_date = Column('CRAWL_DATE', Integer, primary_key=True)
    job_no = Column('JOB_NO', String, primary_key=True)
    total_count = Column('TOTAL_COUNT', Integer)
    sex = Column('SEX', String)
    edu = Column('EDU', String)
    year_range = Column('YEAR_RANGE', String)
    work_exp = Column('WORK_EXP', String)
    lang = Column('LANG', String)
    major = Column('MAJOR', String)
    skill = Column('SKILL', String)
    cert = Column('CERT', String)
    update_date = Column('UPDATE_DATE', Integer)
'''
for text in t.split('\n'):
#     print(text.split(' ')[0].upper())
    print(text.split('=')[0]+'=')

    crawl_date =
    job_no =
    total_count =
    sex =
    edu =
    year_range =
    work_exp =
    lang =
    major =
    skill =
    cert =
    update_date =
=


# class

In [17]:
import sys
sys.path.append('..')

import re
import json
from sqlalchemy.orm import sessionmaker
from sqlalchemy import and_
from datetime import datetime
import pandas as pd
from sqlalchemy.exc import IntegrityError

from model.model import db_connect, create_table, TJobPrim, TRawJobDesc, TRawCompDesc, TJobDesc, TJobAnalysis


In [18]:
db_folder = 'db'
db_name = 'test_db.db'
DB_CONNECTION_STRING = f'sqlite:///{db_folder}/{db_name}'


ANALYSIS_TYPE_LIST = ['sex', 'edu', 'yearRange', 'exp', 'language', 'major', 'skill', 'cert']

class JobDataPreprocessor:
    
    def __init__(self):
        engine = db_connect(DB_CONNECTION_STRING)
        create_table(engine)
        self.Session = sessionmaker(bind=engine)
        
    
    def process(self):
        session = self.Session()#(expire_on_commit=False)
        
        raw_jobs = session.query(TRawJobDesc).all()    
        pk_crawl_date_list = []
        pk_job_no_list = []
        for raw_job in raw_jobs:
            pk_crawl_date_list.append(raw_job.crawl_date)
            pk_job_no_list.append(raw_job.job_no)
        session.close()
        
        for crawl_date, job_no in zip(pk_crawl_date_list, pk_job_no_list):
            
            raw_job = session.query(TRawJobDesc).filter_by(crawl_date=crawl_date, job_no=job_no).first()
            
            crawl_date = raw_job.crawl_date
            job_no = raw_job.job_no
            
            crawled_search_job_json = raw_job.search_job_json
            crawled_search_job_dict = SearchJobJsonProcessor(crawled_search_job_json).process()
            
            crawled_job_desc_json = raw_job.job_desc_json
            crawled_job_desc_dict = JobDescJsonProcessor(crawled_job_desc_json).process()
            
            crawled_job_analysis_json = raw_job.job_analysis_json
            crawled_job_analysis_dict = JobAnalysisJsonProcessor(crawled_job_analysis_json).process()
            
            job_desc_dict, job_analysis_dict = self._format_crawled_data_to_table_column(
                crawl_date, 
                job_no, 
                crawled_search_job_dict, 
                crawled_job_desc_dict, 
                crawled_job_analysis_dict
            )
            
            print(f'{crawl_date}  {job_no}')

            try:
                '''Upsert JOB_DESC data.'''
                job_desc = TJobDesc(**job_desc_dict)
                session.add(job_desc)
                session.commit()
                
            except IntegrityError:
                print(f'>>> JOB_NO: "{job_no}" duplicated. Update data...')
                session.rollback()
                try:
                    session.merge(job_desc)
                    session.commit()
                except Exception as e:
                    print(f'>>> JOB_NO: "{job_no}" duplicated. Update failed.\n"{e}"')
                
            except Exception as e:
                print(f'>>> Error: {e}')
                session.rollback()
                raise
                
            try:           
                job_analysis = TJobAnalysis(**job_analysis_dict)
                session.add(job_analysis)
                session.commit()
                
            except IntegrityError:
                update_date = job_analysis_dict['update_date']
                print(f'>>> JOB_NO: "{job_no}", UPDATE_DATE: "{update_date}" duplicated.')
                session.rollback()
                print('does not insert data, but delete raw data')
                session.delete(raw_job)
                session.commit()
                
            except Exception as e:
                print(f'>>> Error: {e}')
                session.rollback()
                raise
                
            else:
                '''Delete raw data.'''
                print('insert data and delete raw data')
                session.delete(raw_job)
                session.commit()
                
            finally:
#                 session.expunge_all()
                session.close()
        
#         self._vaccum_db()
        print('done')
        
        
    def _vaccum_db(self):
        session = self.Session() 
        session.execute('VACUUM')
        session.close()
                
                
    
    @staticmethod
    def _format_crawled_data_to_table_column(
            crawl_date, 
            job_no, 
            crawled_search_job_dict, 
            crawled_job_desc_dict, 
            crawled_job_analysis_dict):
        job_desc_dict = {}
        job_analysis_dict = {}
        
        job_desc_dict['job_no'] = crawled_search_job_dict['job_no']
        job_desc_dict['job_id'] = crawled_search_job_dict['job_id']
        job_desc_dict['job_name'] = crawled_search_job_dict['job_name']
        job_desc_dict['job_link'] = crawled_search_job_dict['job_link']
        job_desc_dict['job_type'] = crawled_search_job_dict['job_type']
        job_desc_dict['job_role'] = crawled_search_job_dict['job_role']
        job_desc_dict['job_cat'] = crawled_job_desc_dict['job_cat']
        job_desc_dict['apply_count'] = crawled_search_job_dict['apply_count']
        job_desc_dict['need_emp'] = crawled_job_desc_dict['need_emp']
        job_desc_dict['salary_min'] = crawled_search_job_dict['salary_min']
        job_desc_dict['salary_max'] = crawled_search_job_dict['salary_max']
        job_desc_dict['salary_desc'] = crawled_search_job_dict['salary_desc']
        job_desc_dict['salary_type'] = crawled_search_job_dict['salary_type']
        job_desc_dict['accept_role'] = crawled_job_desc_dict['accept_role']
        job_desc_dict['disaccept_role'] = crawled_job_desc_dict['disaccept_role']
        job_desc_dict['edu'] = crawled_search_job_dict['edu']
        job_desc_dict['work_exp'] = crawled_search_job_dict['work_exp']
        job_desc_dict['major'] = crawled_job_desc_dict['major']
        job_desc_dict['lang'] = crawled_job_desc_dict['lang']
        job_desc_dict['local_lang'] = crawled_job_desc_dict['local_lang']
        job_desc_dict['speciality'] = crawled_job_desc_dict['speciality']
        job_desc_dict['skill'] = crawled_job_desc_dict['skill']
        job_desc_dict['cert'] = crawled_job_desc_dict['cert']
        job_desc_dict['driver_license'] = crawled_job_desc_dict['driver_license']
        job_desc_dict['other'] = crawled_job_desc_dict['other']
        job_desc_dict['job_desc'] = crawled_search_job_dict['job_desc']
        job_desc_dict['job_tag'] = crawled_search_job_dict['job_tag']
        job_desc_dict['manage_resp'] = crawled_job_desc_dict['manage_resp']
        job_desc_dict['business_trip'] = crawled_job_desc_dict['business_trip']
        job_desc_dict['work_period'] = crawled_job_desc_dict['work_period']
        job_desc_dict['vacation_policy'] = crawled_job_desc_dict['vacation_policy']
        job_desc_dict['start_work_day'] = crawled_job_desc_dict['start_work_day']
        job_desc_dict['hire_type'] = crawled_job_desc_dict['hire_type']
        job_desc_dict['delegate_recruit'] = crawled_job_desc_dict['delegate_recruit']
        job_desc_dict['job_addr_dist'] = crawled_search_job_dict['job_addr_dist']
        job_desc_dict['landmark_tag'] = crawled_search_job_dict['landmark_tag']
        job_desc_dict['lon'] = crawled_search_job_dict['lon']
        job_desc_dict['lat'] = crawled_search_job_dict['lat']
        job_desc_dict['appear_date'] = crawled_search_job_dict['appear_date']
        
        job_analysis_dict['update_date'] = crawled_job_analysis_dict['update_date']
        job_analysis_dict['job_no'] = job_no
        job_analysis_dict['need_count_min'] = crawled_job_desc_dict['need_count_min']
        job_analysis_dict['need_count_max'] = crawled_job_desc_dict['need_count_max']
        job_analysis_dict['apply_count'] = crawled_job_analysis_dict['apply_count']
        job_analysis_dict['sex'] = crawled_job_analysis_dict['sex']
        job_analysis_dict['edu'] = crawled_job_analysis_dict['edu']
        job_analysis_dict['year_range'] = crawled_job_analysis_dict['year_range']
        job_analysis_dict['work_exp'] = crawled_job_analysis_dict['work_exp']
        job_analysis_dict['lang'] = crawled_job_analysis_dict['lang']
        job_analysis_dict['major'] = crawled_job_analysis_dict['major']
        job_analysis_dict['skill'] = crawled_job_analysis_dict['skill']
        job_analysis_dict['cert'] = crawled_job_analysis_dict['cert']
        job_analysis_dict['crawl_date'] = crawl_date

        return job_desc_dict, job_analysis_dict

In [19]:
class SearchJobJsonProcessor:
    
    def __init__(self, search_job_json: str):
        self.d = json.loads(search_job_json)
        
        
    def process(self):        
        d = self.d
        search_job_dict = {}
        
        job_link = 'https:' + d['link']['job'] if 'job' in d['link'].keys() else ''
        search_job_dict['job_type'] = d['jobType'] # code
        search_job_dict['job_no'] = d['jobNo']
        search_job_dict['job_name'] = d['jobName']
        search_job_dict['job_role'] = d['jobRole'] # code
        search_job_dict['job_addr_dist'] = d['jobAddrNoDesc']
        search_job_dict['job_desc'] = _get_job_description(d)
        search_job_dict['edu'] = d['optionEdu']
        search_job_dict['work_exp'] = d['period'] # int
        search_job_dict['salary_min'] = d['salaryLow'] # int
        search_job_dict['salary_max'] = d['salaryHigh'] # int
        search_job_dict['salary_desc'] = d['salaryDesc']
        search_job_dict['salary_type'] = d['s10'] # code 10:面議 20:論件計酬 30:時薪 50:月薪
        search_job_dict['appear_date'] = d['appearDate'] # int
        search_job_dict['job_tag'] = '、'.join(d['tags'])
        search_job_dict['landmark_tag'] = d['landmark']
        search_job_dict['job_link'] = job_link
        search_job_dict['job_id'] = _get_job_id_from_link(job_link)
        search_job_dict['lon'] = d['lon'][:11]
        search_job_dict['lat'] = d['lat'][:11]
        search_job_dict['apply_count'] = int(d['applyCnt'])
        
        return search_job_dict
    
    
class JobDescJsonProcessor:
    
    def __init__(self, job_desc_json: str):
        self.d = json.loads(job_desc_json)
        
        
    def process(self):
        d = self.d
        job_desc_dict = {}
        
        job_desc_dict['accept_role'] = _get_accept_role_string(d)
        job_desc_dict['disaccept_role'] = _get_disaccept_role_string(d)
        job_desc_dict['major'] = _get_major_string(d)
        job_desc_dict['lang'] = _get_language_string(d)
        job_desc_dict['local_lang'] = _get_local_language_string(d)
        job_desc_dict['speciality'] = _get_specialty_string(d)
        job_desc_dict['skill'] = _get_skill_string(d)
        job_desc_dict['cert'] = _get_certificate_string(d)
        job_desc_dict['driver_license'] = _get_driver_license_string(d)
        job_desc_dict['other'] = _get_other_string(d)
        job_desc_dict['job_cat'] = _get_job_category_string(d)
        job_desc_dict['manage_resp'] = _get_manage_resp_string(d)
        job_desc_dict['business_trip'] = _get_business_trip_string(d)
        job_desc_dict['work_period'] = _get_work_period_string(d)
        job_desc_dict['vacation_policy'] = _get_vacation_policy_string(d)
        job_desc_dict['start_work_day'] = _get_start_working_day_string(d)
        job_desc_dict['hire_type'] = _get_hire_type_string(d)
        job_desc_dict['delegate_recruit'] = _get_delegate_recruit_string(d)
        need_emp_string = _get_need_emp_string(d)
        job_desc_dict['need_emp'] = need_emp_string
        job_desc_dict['need_count_min'] = _get_need_emp_min(need_emp_string)
        job_desc_dict['need_count_max'] = _get_need_emp_max(need_emp_string)
        
        return job_desc_dict
        
        
class JobAnalysisJsonProcessor:
    def __init__(self, job_analysis_json: str):
        self.d = json.loads(job_analysis_json)
        
    
    def process(self):
        d = self.d
        job_analysis_dict = {}
        
        job_analysis_dict['apply_count'] = _get_analysis_apply_count(d)
        job_analysis_dict['update_date'] = _get_update_date(d)
        job_analysis_dict['sex'] = _get_analysis_json_by_type_name(d, 'sex')
        job_analysis_dict['edu'] = _get_analysis_json_by_type_name(d, 'edu')
        job_analysis_dict['year_range'] = _get_analysis_json_by_type_name(d, 'yearRange')
        job_analysis_dict['work_exp'] = _get_analysis_json_by_type_name(d, 'exp')
        job_analysis_dict['lang'] = _get_analysis_json_by_type_name(d, 'language')
        job_analysis_dict['major'] = _get_analysis_json_by_type_name(d, 'major')
        job_analysis_dict['skill'] = _get_analysis_json_by_type_name(d, 'skill')
        job_analysis_dict['cert'] = _get_analysis_json_by_type_name(d, 'cert')
        
        return job_analysis_dict
        
        
        
# search job
def _get_job_description(d):
    desc = d['description']
    return remove_unicode_blank(desc)
    

def _get_job_id_from_link(link):
    try:
        return re.search('job\/(.*)\?', link).group(1)
    except:
        print(f'There is no JOB_ID. (link: {link})')
        return None
    
    
# utils
def join_dict_item_in_list(l: list, key: str):
    dict_item_list = [d[key] for d in l]
    return '、'.join(dict_item_list)


def join_list_element(l: list):
    return '、'.join(l)


def remove_unicode_blank(text):
    text = text.replace('\xa0', '')
    text = text.replace('\u3000', '')
    text = text.replace('\r', '')
    return text


# job desc
def _get_accept_role_string(d):
    l = d['data']['condition']['acceptRole']['role']
    return join_dict_item_in_list(l, 'description')


def _get_disaccept_role_string(d):
    l = d['data']['condition']['acceptRole']['disRole']['disability']
    return join_dict_item_in_list(l, 'type')


def _get_major_string(d):
    l = d['data']['condition']['major']
    return join_list_element(l)


def _get_language_string(d):
    l = d['data']['condition']['language']
    return join_dict_item_in_list(l, 'language')


def _get_local_language_string(d):
    l = d['data']['condition']['localLanguage']
    return join_dict_item_in_list(l, 'language')


def _get_specialty_string(d):
    l = d['data']['condition']['specialty']
    return join_dict_item_in_list(l, 'description')


def _get_skill_string(d):
    l = d['data']['condition']['skill']
    return join_dict_item_in_list(l, 'description')


def _get_certificate_string(d):
    l = d['data']['condition']['certificate']
    return join_list_element(l)


def _get_driver_license_string(d):
    l = d['data']['condition']['driverLicense']
    return join_list_element(l)


def _get_other_string(d):
    return d['data']['condition']['other']


def _get_job_category_string(d):
    l = d['data']['jobDetail']['jobCategory']
    return join_dict_item_in_list(l, 'description')


def _get_manage_resp_string(d):
    return d['data']['jobDetail']['manageResp']


def _get_business_trip_string(d):
    return d['data']['jobDetail']['businessTrip']


def _get_work_period_string(d):
    return d['data']['jobDetail']['workPeriod']


def _get_vacation_policy_string(d):
    return d['data']['jobDetail']['vacationPolicy']


def _get_start_working_day_string(d):
    return d['data']['jobDetail']['startWorkingDay']


def _get_hire_type_string(d):
    ''' code
    0: 公司自聘
    1: 代徵
    '''
    return str(d['data']['jobDetail']['hireType'])


def _get_delegate_recruit_string(d):
    return d['data']['jobDetail']['delegatedRecruit']


def _get_need_emp_string(d):
    return d['data']['jobDetail']['needEmp']


def _get_need_emp_min(need_emp_string):
    result = re.findall('(\d+)', need_emp_string)
    if result:
        return result[0]
    else:
        return 1
    
def _get_need_emp_max(need_emp_string):
    result = re.findall('(\d+)', need_emp_string)
    if result:
        return result[-1]
    else:
        return 1


# job analysis
def _convert_analysis_dict_to_format_json(raw_analysis_dict):
    stat_dict = {key: raw_analysis_dict[key] for key in raw_analysis_dict.keys() if key.isdigit()}
    stat_df = pd.DataFrame.from_dict(stat_dict)
    format_json = stat_df.to_json(force_ascii=False)
    return format_json


def _get_analysis_json_by_type_name(raw_dict, key):
    raw_analysis_dict = raw_dict[key]
    analysis_json = _convert_analysis_dict_to_format_json(raw_analysis_dict)
    return analysis_json


def _get_analysis_apply_count(raw_dict):
    ''' TODO: use DFS to find first 'total' value. Same thing for "_get_update_date"
    '''
    for type_name in ANALYSIS_TYPE_LIST:
        if 'total' in raw_dict[type_name].keys():
            total = raw_dict[type_name]['total']
            return int(total)
    return -1
    

def _get_update_date(raw_dict):
    for type_name in ANALYSIS_TYPE_LIST:
        if 'update_time' in raw_dict[type_name].keys():
            update_time_string = raw_dict[type_name]['update_time']
            update_date = datetime.strptime(update_time_string, '%Y-%m-%d %H:%M:%S')
            update_date = int(update_date.strftime('%Y%m%d'))
            return update_date
    return -1

In [20]:
JobDataPreprocessor().process()

>>> DB_CONNECTION_STRING: sqlite:///db/test_db.db
20201107  11927515
insert data and delete raw data
20201107  11612382
insert data and delete raw data
20201107  11909807
insert data and delete raw data
20201107  11927472
insert data and delete raw data
20201107  11860911
insert data and delete raw data
20201107  11854233
insert data and delete raw data
20201107  7468390
insert data and delete raw data
20201107  11627220
insert data and delete raw data
20201107  10755645
insert data and delete raw data
20201107  11118467
insert data and delete raw data
20201107  11312529
insert data and delete raw data
20201107  11572853
insert data and delete raw data
20201107  11073371
insert data and delete raw data
20201107  10711594
insert data and delete raw data
20201107  11004075
insert data and delete raw data
20201107  11647900
insert data and delete raw data
20201107  11769088
insert data and delete raw data
20201107  10935172
insert data and delete raw data
20201107  11774852
insert data an

20201107  11774429
insert data and delete raw data
20201107  7048666
insert data and delete raw data
20201107  11624547
insert data and delete raw data
20201107  11487977
insert data and delete raw data
20201107  11528892
insert data and delete raw data
20201107  11616412
insert data and delete raw data
20201107  8310733
insert data and delete raw data
20201107  10890284
insert data and delete raw data
20201107  11813811
insert data and delete raw data
20201107  7010552
insert data and delete raw data
20201107  11340857
insert data and delete raw data
20201107  11059367
insert data and delete raw data
20201107  1564025
insert data and delete raw data
20201107  11383146
insert data and delete raw data
20201107  11525191
insert data and delete raw data
20201107  11262301
insert data and delete raw data
20201107  10057904
insert data and delete raw data
20201107  11701833
insert data and delete raw data
20201107  11620817
insert data and delete raw data
20201107  10145126
insert data and 

20201107  5874383
insert data and delete raw data
20201107  9190647
insert data and delete raw data
20201107  11719177
insert data and delete raw data
20201107  5509691
insert data and delete raw data
20201107  8540151
insert data and delete raw data
20201107  11429524
insert data and delete raw data
20201107  5874394
insert data and delete raw data
20201107  11204922
insert data and delete raw data
20201107  11749066
insert data and delete raw data
20201107  8288227
insert data and delete raw data
20201107  10737653
insert data and delete raw data
20201107  11855737
insert data and delete raw data
20201107  11908282
insert data and delete raw data
20201107  11683176
insert data and delete raw data
20201107  11918794
insert data and delete raw data
20201107  11753822
insert data and delete raw data
20201107  9365514
insert data and delete raw data
20201107  9641336
insert data and delete raw data
20201107  11774964
insert data and delete raw data
20201107  11845762
insert data and dele

insert data and delete raw data
20201107  11502004
insert data and delete raw data
20201107  11712556
insert data and delete raw data
20201107  11451297
insert data and delete raw data
20201107  7448630
insert data and delete raw data
20201107  11760961
insert data and delete raw data
20201107  11013249
insert data and delete raw data
20201107  11675986
insert data and delete raw data
20201107  11544623
insert data and delete raw data
20201107  11483501
insert data and delete raw data
20201107  11540788
insert data and delete raw data
20201107  11359916
insert data and delete raw data
20201107  11451285
insert data and delete raw data
20201107  11295474
insert data and delete raw data
20201107  8228022
insert data and delete raw data
20201107  7448699
insert data and delete raw data
20201107  11863064
insert data and delete raw data
20201107  11063014
insert data and delete raw data
20201107  4311373
insert data and delete raw data
20201107  10747229
insert data and delete raw data
202

insert data and delete raw data
20201107  4524869
insert data and delete raw data
20201107  11323014
insert data and delete raw data
20201107  11573712
insert data and delete raw data
20201107  8078344
insert data and delete raw data
20201107  4492696
insert data and delete raw data
20201107  7949799
insert data and delete raw data
20201107  11001364
insert data and delete raw data
20201107  8587843
insert data and delete raw data
20201107  11904105
insert data and delete raw data
20201107  8807550
insert data and delete raw data
20201107  4525190
insert data and delete raw data
20201107  11839080
insert data and delete raw data
20201107  10039852
insert data and delete raw data
20201107  8912074
insert data and delete raw data
20201107  9971010
insert data and delete raw data
20201107  11102711
insert data and delete raw data
20201107  7525031
insert data and delete raw data
20201107  6428214
insert data and delete raw data
20201107  11779989
insert data and delete raw data
20201107  

>>> JOB_NO: "9528028" duplicated. Update data...
insert data and delete raw data
20201113  10937888
>>> JOB_NO: "10937888" duplicated. Update data...
insert data and delete raw data
20201113  9561662
>>> JOB_NO: "9561662" duplicated. Update data...
insert data and delete raw data
20201113  11860911
>>> JOB_NO: "11860911" duplicated. Update data...
insert data and delete raw data
20201113  10935172
>>> JOB_NO: "10935172" duplicated. Update data...
insert data and delete raw data
20201113  11217856
>>> JOB_NO: "11217856" duplicated. Update data...
insert data and delete raw data
20201113  11647900
>>> JOB_NO: "11647900" duplicated. Update data...
insert data and delete raw data
20201113  11927336
insert data and delete raw data
20201113  10242837
>>> JOB_NO: "10242837" duplicated. Update data...
insert data and delete raw data
20201113  4242185
>>> JOB_NO: "4242185" duplicated. Update data...
insert data and delete raw data
20201113  8539635
>>> JOB_NO: "8539635" duplicated. Update data.

,0,1,2,3,4,5,6,7,8,9
cert,4001001002,4002036028,4002035045,4003009017,4002036027,4006001045,4002040001,4012002003,4001001018,4023001001
certName,TOEFL,軟體專案管理,TQC專業程式設計工程師,氬氣鎢極電銲技術士,專案管理,電腦稽核軟體應用師JCCP,ERP規劃師,國際行銷初級人才認證檢定,BULATS(劍橋博思國際職場英檢),國際專案管理師PMP
count,2,1,1,1,1,1,1,1,1,1
percent,18.18,9.09,9.09,9.09,9.09,9.09,9.09,9.09,9.09,9.09


In [64]:
need_emp = '1人'
need_emp = '10~20人'
# need_emp = '10人'
# need_emp = '不限'

def _get_need_emp_min():
    need_emp = 1
    
result = re.findall('(\d+)', need_emp)
if result:
    print(result[-1])
else:
    print(1)

20


In [1]:
json_string = '''{'jobType': '2', 'jobNo': '11612382', 'jobName': '網頁爬蟲工程師', 'jobNameSnippet': "網頁<em class='b-txt--highlight'>爬蟲</em>工程師", 'jobRole': '1', 'jobRo': '1', 'jobAddrNo': '6001008007', 'jobAddrNoDesc': '台中市西屯區', 'jobAddress': '市政北二路236號35樓之1', 'description': '網站[[[爬蟲]]]、資料輸出、API串接', 'optionEdu': '大學', 'period': '00', 'periodDesc': '經歷不拘', 'applyCnt': '00001', 'applyDesc': '0~5人應徵', 'custNo': '130000000012043', 'custName': '凡谷興業有限公司', 'coIndustry': '1001001006', 'coIndustryDesc': '其它軟體及網路相關業', 'salaryLow': '0030000', 'salaryHigh': '0060000', 'salaryDesc': '月薪30,000~60,000元', 's10': '50', 'appearDate': '20201104', 'appearDateDesc': '11/04', 'optionZone': '2101376', 'isApply': '0', 'applyDate': '', 'isSave': '0', 'descSnippet': "網站<em class='b-txt--highlight'>爬蟲</em>、資料輸出、API串接", 'tags': [], 'landmark': '', 'link': {'applyAnalyze': '//www.104.com.tw/jobs/apply/analysis/6ww66?channel=104rpt&jobsource=apply_analyze', 'job': '//www.104.com.tw/job/6ww66?jobsource=n104bank2', 'cust': '//www.104.com.tw/company/1a2x6bi8ln?jobsource=n104bank2'}, 'jobsource': '', 'jobNameRaw': '網頁爬蟲工程師', 'custNameRaw': '凡谷興業有限公司', 'lon': '120.6376087', 'lat': '24.1637161'}'''


In [2]:
import json

In [3]:
json.loads(json_string)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)